# Waktu Proses ONNX

Di notebook ini, kami akan menunjukkan cara menggunakan ONNX Runtime untuk mempercepat inferensi model yang dilatih di PyTorch. Selain itu, kami akan menggunakan ONNX untuk mengkuantisasi model hingga presisi int8 guna lebih meningkatkan kinerja dengan mengurangi jejak memori. Kami akan melatih model sederhana pada kumpulan data MNIST dan kemudian mengonversinya ke format ONNX. Kami kemudian akan menggunakan ONNX Runtime untuk mempercepat inferensi model. Terakhir, kami akan mengkuantisasi model hingga presisi int8

## Atur Waktu Proses ONNX

Pertama, instal torch, torchvision, onnx dan onnxruntime. Kemudian, impor modul yang diperlukan

In [1]:
%pip install torch torchvision
%pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib
import numpy as np
import torch.onnx
import onnx
import onnxruntime
from onnxruntime.quantization import quantize_dynamic, quantize_static, CalibrationDataReader, QuantType

## Model Kereta Api

Kami akan melatih model CNN sederhana pada dataset MNIST.

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

MODEL_DIR = pathlib.Path("./onnx_models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(model.state_dict(), MODEL_DIR / "original_model.p")

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 57.1MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 2.04MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

100%|██████████| 1.65M/1.65M [00:00<00:00, 15.0MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.79MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.330969
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.370600
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.184278
Train Epoch: 1 [96/60000 (0%)]	Loss: 2.031038
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.078423
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.083939
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.688972
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.777475
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.891660
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.526311
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.654433
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.453821
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.422168
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.356842
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.251338
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.603687
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.367763
Train Epoch: 1 [544/60000 (1%)]	Loss: 1.115706
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.355937
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.381475
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.308299
Train Epoch: 1 [67

## Ekspor ke ONNX

Setelah pelatihan, ekspor model ke format ONNX.

In [4]:
x, _ = next(iter(train_loader))
torch.onnx.export(model,
                  x,
                  MODEL_DIR / "mnist_model.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},
                                'output' : {0 : 'batch_size'}})

## Jalankan Inferensi dan Uji Kemiripan

Selanjutnya, validasi model yang dikonversi dengan menjalankan inferensi dan membandingkan hasilnya dengan model PyTorch.

In [5]:
torch_out = model(x)

onnx_model = onnx.load(MODEL_DIR / "mnist_model.onnx")
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession(MODEL_DIR / "mnist_model.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


## Quantization

### Kuantisasi Dinamis

Kuantisasi dinamis menghitung parameter yang akan dikuantisasi untuk aktivasi secara dinamis. Perhitungan ini meningkatkan akurasi model namun juga meningkatkan biaya inferensi.

In [6]:
!python -m onnxruntime.quantization.preprocess --input {MODEL_DIR / "mnist_model.onnx"} --output {MODEL_DIR / "mnist_model_processed.onnx"}

In [7]:
model_fp32 = MODEL_DIR / "mnist_model_processed.onnx"
model_quant = MODEL_DIR / "mnist_model_quant.onnx"
quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

### Bandingkan Ukuran

Mari kita bandingkan ukuran model aslinya, model terkuantisasi

In [8]:
%ls -lh {MODEL_DIR}

total 280K
-rw-r--r-- 1 root root 82K Jan  3 15:14 mnist_model.onnx
-rw-r--r-- 1 root root 82K Jan  3 15:14 mnist_model_processed.onnx
-rw-r--r-- 1 root root 26K Jan  3 15:14 mnist_model_quant.onnx
-rw-r--r-- 1 root root 82K Jan  3 15:14 original_model.p


### Bandingkan Akurasi

Mari kita bandingkan keakuratan model onnx yang dikonversi dan model terkuantisasi. Keakuratan model terkuantisasi harus mendekati model aslinya

In [9]:
def test_onnx(model_name, data_loader):
    onnx_model = onnx.load(model_name)
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(model_name)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(data)}
        output = ort_session.run(None, ort_inputs)[0]
        output = torch.from_numpy(output)
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)

    return 100. * correct / len(data_loader.dataset)

acc = test_onnx(MODEL_DIR / "mnist_model.onnx", test_loader)
print(f"Accuracy of the original model is {acc}%")

qacc = test_onnx(MODEL_DIR / "mnist_model_quant.onnx", test_loader)
print(f"Accuracy of the quantized model is {qacc}%")


Accuracy of the original model is 96.47%
Accuracy of the quantized model is 96.53%


## Kuantisasi Statis

Untuk metode kuantisasi statis, parameter dikuantisasi terlebih dahulu menggunakan dataset kalibrasi. Metode ini lebih cepat dibandingkan kuantisasi dinamis namun akurasinya lebih rendah. Oleh karena itu, kumpulan data kalbrasi perlu dibuat menggunakan kelas `CalibrationDataReader`.

In [10]:
class QuantDR(CalibrationDataReader):
    def __init__(self, torch_data_loader, input_name):
        self.torch_data_loader = torch_data_loader
        self.input_name = input_name
        self.datasize = len(torch_data_loader)
        self.enum_data = iter(torch_data_loader)

    def to_numpy(self, tensor):
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

    def get_next(self):
        batch = next(self.enum_data, None)
        if batch is not None:
          return {self.input_name: self.to_numpy(batch[0])}
        else:
          return None

    def rewind(self):
        self.enum_data = iter(self.torch_data_loader)

calibration_data = QuantDR(train_loader, ort_session.get_inputs()[0].name)
model__static_quant = MODEL_DIR / "mnist_model_static_quant.onnx"
static_quant_model = quantize_static(model_fp32, model__static_quant, calibration_data, weight_type=QuantType.QInt8)

### Bandingkan Ukuran

Mari kita bandingkan ukuran model asli dan model terkuantisasi

In [11]:
%ls -lh {MODEL_DIR}

total 308K
-rw-r--r-- 1 root root 82K Jan  3 15:14 mnist_model.onnx
-rw-r--r-- 1 root root 82K Jan  3 15:14 mnist_model_processed.onnx
-rw-r--r-- 1 root root 26K Jan  3 15:14 mnist_model_quant.onnx
-rw-r--r-- 1 root root 25K Jan  3 15:15 mnist_model_static_quant.onnx
-rw-r--r-- 1 root root 82K Jan  3 15:14 original_model.p


### Bandingkan Akurasi

Mari kita bandingkan keakuratan model onnx yang dikonversi dan model terkuantisasi. Keakuratan model terkuantisasi harus mendekati model aslinya

In [12]:
static_qacc = test_onnx(model__static_quant, test_loader)
print(f"Accuracy of the static quantized model is {static_qacc}%")

Accuracy of the static quantized model is 96.51%
